# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface = await new_transport_interface("0.0.0.0", 12345)
tiface.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fd92721b7f0>()]>

# Filter layer setup

In [4]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [5]:
[m for m in dir(fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Filter!

In [6]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()

Get the number of milliseconds between calling `get_notifications` on the contained `TransportInterface`. This can also be assigned to at any time. Lower values can cause performance issues, while higher values add latency for anything involving filter notifications.

In [7]:
fiface.notif_poll_ms

30

In [8]:
fiface.notif_poll_ms = 100

## Filter layer operation

### As a client, connect to server

In [9]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient7910", client_version: "0.0.666" }

In [10]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=chococat_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(157.230.134.224:2016), action: Connect { name: "FilterNotebookClient7910", client_version: "0.0.666" } }

In [11]:
await fiface.command_response(filter_cmd)

Accepted

In [12]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "c-csEg8dNrfRI1Ql", server_version: "0.3.5" } }]

In [13]:
new_response_code = _[0]
new_response_code

NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "c-csEg8dNrfRI1Ql", server_version: "0.3.5" } }

In [14]:
cookie = new_response_code.response_code.cookie
cookie

'c-csEg8dNrfRI1Ql'

### Convenience function

In [15]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await fiface.command_response(filter_cmd)

### Join a room

In [16]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Accepted

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [17]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: JoinedRoom { room_name: "general" } },
 NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: BadRequest { error_msg: "cannot join game because in-game" } }]